In [2]:
import glob
import os
from os import getcwd

In [ ]:
def readClass(class_dir):
    classes = {}
    f = open(class_dir, 'r')
    data = f.readlines()
    classNum = 0
    for d in data:
        classes[classNum] = d.replace('\n', '')
        classNum += 1
    
    return classes

In [ ]:
# get all the images in img_path
def getImagesInDir(img_path):
    image_list = []
    for filename in glob.glob(img_path + '/*.jpg'):
        image_list.append(filename)

    return image_list

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

# @xml_path: where all xml exists
# @output_path: where you want to store output
# @image_path: list generate from getImagesInDir()
# @class_dir: dir of classes.names
def convert_annotation(xml_path, output_path, image_path, class_dir):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]
    classes = readClass(class_dir)
    classes_val = list(classes.values())
    classes_key = list(classes.keys())
    
    if os.path.exists(xml_path + '/' + basename_no_ext + '.xml'):
        in_file = open(xml_path + '/' + basename_no_ext + '.xml')
        out_file = open(output_path + basename_no_ext + '.txt', 'w')
        tree = et.parse(in_file)
        root = tree.getroot()
        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)

        for obj in root.iter('object'):
            difficult = obj.find('difficult').text
            cls = obj.find('name').text
            if cls not in classes_val or int(difficult)==1:
                continue
            cls_id = classes_key[classes_val.index(cls)]
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = convert((w,h), b)
            out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')